<a href="https://colab.research.google.com/github/acate/llm_train/blob/main/examples/language_modeling_from_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

If you're opening this Notebook on colab, you will probably need to install 🤗 Transformers and 🤗 Datasets. Uncomment the following cell and run it.

In [1]:
! pip install datasets

If you're opening this notebook locally, make sure your environment has an install from the last version of those libraries.

To be able to share your model with the community and generate results like the one shown in the picture below via the inference API, there are a few more steps to follow.

First you have to store your authentication token from the Hugging Face website (sign up [here](https://huggingface.co/join) if you haven't already!) then execute the following cell and input your username and password:

In [65]:
from huggingface_hub import notebook_login

notebook_login()

Then you need to install Git-LFS. Uncomment the following instructions:

In [66]:
!apt install git-lfs

Make sure your version of Transformers is at least 4.11.0 since the functionality was introduced in that version:

In [67]:
!pip install transformers==4.26.0 # This version is compatible with accelerate v. 0.20.0, below
import transformers

print(transformers.__version__)

You can find a script version of this notebook to fine-tune your model in a distributed fashion using multiple GPUs or TPUs [here](https://github.com/huggingface/transformers/tree/master/examples/language-modeling).

We also quickly upload some telemetry - this tells us which examples and software versions are getting used so we know where to prioritize our maintenance efforts. We don't collect (or care about) any personally identifiable information, but if you'd prefer not to be counted, feel free to skip this step or delete this cell entirely.

In [68]:
from transformers.utils import send_example_telemetry

send_example_telemetry("language_modeling_from_scratch_notebook", framework="pytorch")

# Train a language model

In this notebook, we'll see how to train a [🤗 Transformers](https://github.com/huggingface/transformers) model on a language modeling task. We will cover two types of language modeling tasks which are:

- Causal language modeling: the model has to predict the next token in the sentence (so the labels are the same as the inputs shifted to the right). To make sure the model does not cheat, it gets an attention mask that will prevent it to access the tokens after token i when trying to predict the token i+1 in the sentence.

![Widget inference representing the causal language modeling task](https://github.com/huggingface/notebooks/blob/main/examples/images/causal_language_modeling.png?raw=1)

- Masked language modeling: the model has to predict some tokens that are masked in the input. It still has access to the whole sentence, so it can use the tokens before and after the tokens masked to predict their value.

We will see how to easily load and preprocess the dataset for each one of those tasks, and how to use the `Trainer` API to train a model on it.

This notebooks assumes you have trained a tokenizer on the corpus you are using, see the [How to train a tokenizer](https://github.com/huggingface/notebooks/blob/master/examples/tokenizer_training.ipynb) notebook ([open in colab](https://colab.research.google.com/github/huggingface/notebooks/blob/master/examples/tokenizer_training.ipynb)).

A script version of this notebook you can directly run on a distributed environment or on TPU is available in our [examples folder](https://github.com/huggingface/transformers/tree/master/examples).

## Preparing the dataset

For each of those tasks, we will use the [Wikitext 2]() dataset as an example. You can load it very easily with the 🤗 Datasets library.

In [69]:
from datasets import load_dataset
datasets = load_dataset('wikitext', 'wikitext-2-raw-v1')

You can replace the dataset above with any dataset hosted on [the hub](https://huggingface.co/datasets) or use your own files. Just uncomment the following cell and replace the paths with values that will lead to your files:

In [ ]:
# datasets = load_dataset("text", data_files={"train": path_to_train.txt, "validation": path_to_validation.txt}

You can also load datasets from a csv or a JSON file, see the [full documentation](https://huggingface.co/docs/datasets/loading_datasets.html#from-local-files) for more information.

To access an actual element, you need to select a split first, then give an index:

In [70]:
datasets["train"][10]

{'text': ' The game \'s battle system , the BliTZ system , is carried over directly from Valkyira Chronicles . During missions , players select each unit using a top @-@ down perspective of the battlefield map : once a character is selected , the player moves the character around the battlefield in third @-@ person . A character can only act once per @-@ turn , but characters can be granted multiple turns at the expense of other characters \' turns . Each character has a field and distance of movement limited by their Action Gauge . Up to nine characters can be assigned to a single mission . During gameplay , characters will call out if something happens to them , such as their health points ( HP ) getting low or being knocked out by enemy attacks . Each character has specific " Potentials " , skills unique to each character . They are divided into " Personal Potential " , which are innate skills that remain unaltered unless otherwise dictated by the story and can either help or impede

To get a sense of what the data looks like, the following function will show some examples picked randomly in the dataset.

In [71]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [72]:
show_random_elements(datasets["train"])

,text
0,"Politically , Ireland is divided between the Republic of Ireland ( officially named Ireland ) , which covers five @-@ sixths of the island , and Northern Ireland , which is part of the United Kingdom , in the northeast of the island . In 2011 the population of Ireland was about 6 @.@ 4 million , ranking it the second @-@ most populous island in Europe after Great Britain . Just under 4 @.@ 6 million live in the Republic of Ireland and just over 1 @.@ 8 million live in Northern Ireland . \n"
1,"The Great Coastal hurricane of 1806 was first noted far east of the Lesser Antilles on 17 August . Weather historian David M. Ludlum followed the disturbance 's track to the Bahamas by 19 August ; intense winds persisted until 21 August , however , approximately 150 mi ( 240 km ) east of the Bahamian island of Eleuthera . Steering currents brought the storm northward , and it approached Charleston , South Carolina on 22 August , where a generally easterly flow preceded the storm indicated its passage far east of the city . The hurricane made landfall at the mouth of the Cape Fear River in North Carolina later that day , though the earliest impacts from the storm started several days earlier , with gusts initially toward the northeast but later curving southwestward . Reports of similar wind shifts throughout the region suggested that the gale persisted , stationary , for several hours . It eventually moved back out to sea while south of Norfolk , Virginia , departing the region on 24 August . The hurricane maintained 1 @-@ minute maximum sustained winds of 110 mph ( 175 km / h ) while offshore , equivalent to a Category 2 system on the Saffir – Simpson hurricane wind scale . While offshore New England , the gale featured a swath of winds 90 mi ( 150 km ) wide , and was last observed just south of Nova Scotia on 25 August slightly weaker , with sustained winds of 75 mph ( 120 km / h ) . \n"
2,
3,
4,
5,= = Service history = = \n
6,
7,= = Life = = \n
8,= = Music video and live performances = = \n
9,"There were many reports of tropical storm @-@ force winds from ships in the north Atlantic . One ship , with the call sign GBSA , encountered Tanya 's winds twice and reported the strongest winds from any ship , 71 mph ( 112 km / h ) . \n"


As we can see, some of the texts are a full paragraph of a Wikipedia article while others are just titles or empty lines.

## Causal Language modeling

For causal language modeling (CLM) we are going to take all the texts in our dataset and concatenate them after they are tokenized. Then we will split them in examples of a certain sequence length. This way the model will receive chunks of contiguous text that may look like:
```
part of text 1
```
or
```
end of text 1 [BOS_TOKEN] beginning of text 2
```
depending on whether they span over several of the original texts in the dataset or not. The labels will be the same as the inputs, shifted to the left.

We will use the [`gpt2`](https://huggingface.co/gpt2) architecture for this example. You can pick any of the checkpoints listed [here](https://huggingface.co/models?filter=causal-lm) instead. For the tokenizer, you can replace the checkpoint by the one you trained yourself.

In [73]:
model_checkpoint = "gpt2"
tokenizer_checkpoint = "sgugger/gpt2-like-tokenizer"

To tokenize all our texts with the same vocabulary that was used when training the model, we have to download a pretrained tokenizer. This is all done by the `AutoTokenizer` class:

In [74]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(tokenizer_checkpoint)



/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
loading file vocab.json from cache at /root/.cache/huggingface/hub/models--sgugger--gpt2-like-tokenizer/snapshots/a21f1b3bb12076a9a82577013e2e5fa962446263/vocab.json
loading file merges.txt from cache at /root/.cache/huggingface/hub/models--sgugger--gpt2-like-tokenizer/snapshots/a21f1b3bb12076a9a82577013e2e5fa962446263/merges.txt
loading file tokenizer.json from cache at /root/.cache/huggingface/hub/models--sgugger--gpt2-like-tokenizer/snapshots/a21f1b3bb12076a9a82577013e2e5fa962446263/tokenizer.json
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at /root/.cache/huggingface/hub/models--sgugger--gpt2-like-tokenizer/snapshots/a21f1b3bb12076a9a82577013e2e5fa96

We can now call the tokenizer on all our texts. This is very simple, using the [`map`](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasets.Dataset.map) method from the Datasets library. First we define a function that call the tokenizer on our texts:

In [75]:
def tokenize_function(examples):
    return tokenizer(examples["text"])

Then we apply it to all the splits in our `datasets` object, using `batched=True` and 4 processes to speed up the preprocessing. We won't need the `text` column afterward, so we discard it.

In [76]:
tokenized_datasets = datasets.map(tokenize_function, batched=True, num_proc=4, remove_columns=["text"])

If we now look at an element of our datasets, we will see the text have been replaced by the `input_ids` the model will need:

In [77]:
tokenized_datasets["train"][1]

{'input_ids': [238, 8576, 9441, 2987, 238, 252],
 'attention_mask': [1, 1, 1, 1, 1, 1]}

Now for the harder part: we need to concatenate all our texts together then split the result in small chunks of a certain `block_size`. To do this, we will use the `map` method again, with the option `batched=True`. This option actually lets us change the number of examples in the datasets by returning a different number of examples than we got. This way, we can create our new samples from a batch of examples.

First, we grab the maximum length our model was pretrained with. This might be a big too big to fit in your GPU RAM, so here we take a bit less at just 128.

In [78]:
# block_size = tokenizer.model_max_length
block_size = 128

Then we write the preprocessing function that will group our texts:

In [79]:
def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
        # customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

First note that we duplicate the inputs for our labels. This is because the model of the 🤗 Transformers library apply the shifting to the right, so we don't need to do it manually.

Also note that by default, the `map` method will send a batch of 1,000 examples to be treated by the preprocessing function. So here, we will drop the remainder to make the concatenated tokenized texts a multiple of `block_size` every 1,000 examples. You can adjust this behavior by passing a higher batch size (which will also be processed slower). You can also speed-up the preprocessing by using multiprocessing:

In [80]:
lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=4,
)

And we can check our datasets have changed: now the samples contain chunks of `block_size` contiguous tokens, potentially spanning over several of our original texts.

In [81]:
tokenizer.decode(lm_datasets["train"][1]["input_ids"])

' the " Nameless ", a penal military unit serving the nation of Gallia during the Second Europan War who perform secret black operations and are pitted against the Imperial unit " Calamaty Raven ". \n The game began development in 2010, carrying over a large portion of the work done on Valkyria Chronicles II. While it retained the standard features of the series, it also underwent multiple adjustments, such as making the game more forgiving for series newcomers. Character designer Raita Honjou and composer Hitoshi Sakimoto both returned from previous entries, along with Valkyria Chronicles II director Takeshi Ozawa. A large'

Now that the data has been cleaned, we're ready to instantiate our `Trainer`. First we create the model using the same config as our checkpoint, but initialized with random weights:

In [82]:
from transformers import AutoConfig, AutoModelForCausalLM

config = AutoConfig.from_pretrained(model_checkpoint)
model = AutoModelForCausalLM.from_config(config)

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--gpt2/snapshots/607a30d783dfa663caf39e06633721c8d4cfcd7e/config.json
Model config GPT2Config {
  "_name_or_path": "gpt2",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 50
    }
  },
  "transformers_vers

And we will needsome `TrainingArguments`:

In [84]:
from transformers import Trainer, TrainingArguments

In [85]:
#!pip install transformers==4.26.0
!pip install accelerate==0.20.0
from transformers import Trainer, TrainingArguments
#import torch
import accelerate
print(f"accelerate version: {accelerate.__version__}")
print(f"transformers version: {transformers.__version__}")

training_args = TrainingArguments(
    f"{model_checkpoint}-wikitext2",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    push_to_hub=True
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


The last argument to setup everything so we can push the model to the [Hub](https://huggingface.co/models) regularly during training. Remove it if you didn't follow the installation steps at the top of the notebook. If you want to save your model locally in a name that is different than the name of the repository it will be pushed, or if you want to push your model under an organization and not your name space, use the `hub_model_id` argument to set the repo name (it needs to be the full name, including your namespace: for instance `"sgugger/gpt-finetuned-wikitext2"` or `"huggingface/gpt-finetuned-wikitext2"`).

We pass along all of those to the `Trainer` class:

In [87]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_datasets["train"],
    eval_dataset=lm_datasets["validation"],
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'Repository' (from 'huggingface_hub.repository') is deprecated and will be removed from version '1.0'. Please prefer the http-based alternatives instead. Given its large adoption in legacy code, the complete removal is only planned on next major release.
For more details, please read https://huggingface.co/docs/huggingface_hub/concepts/git_vs_http.
  warnings.warn(warning_message, FutureWarning)
/content/gpt2-wikitext2 is already a clone of https://huggingface.co/acate/gpt2-wikitext2. Make sure you pull the latest changes with `repo.git_pull()`.


And we can train our model:

In [88]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 17991
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 6747
  Number of trainable parameters = 124439808


Epoch,Training Loss,Validation Loss
1,6.563100,6.471740
2,6.194700,6.203093
3,6.018900,6.111773


Saving model checkpoint to gpt2-wikitext2/checkpoint-500
Configuration saved in gpt2-wikitext2/checkpoint-500/config.json
Configuration saved in gpt2-wikitext2/checkpoint-500/generation_config.json
Model weights saved in gpt2-wikitext2/checkpoint-500/pytorch_model.bin
Saving model checkpoint to gpt2-wikitext2/checkpoint-1000
Configuration saved in gpt2-wikitext2/checkpoint-1000/config.json
Configuration saved in gpt2-wikitext2/checkpoint-1000/generation_config.json
Model weights saved in gpt2-wikitext2/checkpoint-1000/pytorch_model.bin
Saving model checkpoint to gpt2-wikitext2/checkpoint-1500
Configuration saved in gpt2-wikitext2/checkpoint-1500/config.json
Configuration saved in gpt2-wikitext2/checkpoint-1500/generation_config.json
Model weights saved in gpt2-wikitext2/checkpoint-1500/pytorch_model.bin
Saving model checkpoint to gpt2-wikitext2/checkpoint-2000
Configuration saved in gpt2-wikitext2/checkpoint-2000/config.json
Configuration saved in gpt2-wikitext2/checkpoint-2000/generat

TrainOutput(global_step=6747, training_loss=6.392105131939426, metrics={'train_runtime': 798.1369, 'train_samples_per_second': 67.624, 'train_steps_per_second': 8.453, 'total_flos': 3525678710784000.0, 'train_loss': 6.392105131939426, 'epoch': 3.0})

Once the training is completed, we can evaluate our model and get its perplexity on the validation set like this:

In [48]:
import math
eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

***** Running Evaluation *****
  Num examples = 2009
  Batch size = 8


The perplexity is still quite high since for this demo we trained on a small dataset for a small number of epochs. For a real LM training, you  would need a larger dataset and more epochs.

You can now upload the result of the training to the Hub, just execute this instruction:

In [89]:
trainer.push_to_hub()
tokenizer.push_to_hub("acate/gpt2-wikitext2")

Saving model checkpoint to gpt2-wikitext2
Configuration saved in gpt2-wikitext2/config.json
Configuration saved in gpt2-wikitext2/generation_config.json
Model weights saved in gpt2-wikitext2/pytorch_model.bin


Upload file pytorch_model.bin:   0%|          | 1.00/487M [00:00<?, ?B/s]

Upload file runs/May23_01-36-26_1969c3de4f38/events.out.tfevents.1716428334.1969c3de4f38.1280.9:   0%|        …

To https://huggingface.co/acate/gpt2-wikitext2
   df7c4a4..2391450  main -> main

   df7c4a4..2391450  main -> main

Dropping the following result as it does not have all the necessary fields:
{'task': {'name': 'Causal Language Modeling', 'type': 'text-generation'}}
To https://huggingface.co/acate/gpt2-wikitext2
   2391450..8ad4091  main -> main

   2391450..8ad4091  main -> main

tokenizer config file saved in gpt2-wikitext2/tokenizer_config.json
Special tokens file saved in gpt2-wikitext2/special_tokens_map.json
Uploading the following files to acate/gpt2-wikitext2: tokenizer.json,tokenizer_config.json,special_tokens_map.json,merges.txt,vocab.json


CommitInfo(commit_url='https://huggingface.co/acate/gpt2-wikitext2/commit/5fa6e57a303ccc43fa97c37d6eda22a9a8ffec6a', commit_message='Upload tokenizer', commit_description='', oid='5fa6e57a303ccc43fa97c37d6eda22a9a8ffec6a', pr_url=None, pr_revision=None, pr_num=None)

You can now share this model with all your friends, family, favorite pets: they can all load it with the identifier `"your-username/the-name-you-picked"` so for instance:

```python
from transformers import AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained("sgugger/my-awesome-model")
```

## Masked language modeling

For masked language modeling (MLM) we are going to use the same preprocessing as before for our dataset with one additional step: we will randomly mask some tokens (by replacing them by `[MASK]`) and the labels will be adjusted to only include the masked tokens (we don't have to predict the non-masked tokens). If you use a tokenizer you trained yourself, make sure the `[MASK]` token is among the special tokens you passed during training!

We will use the [`bert-base-cased`](https://huggingface.co/bert-based-cased) model for this example. You can pick any of the checkpoints listed [here](https://huggingface.co/models?filter=masked-lm) instead. For the tokenizer, replace the checkpoint by the one you trained.

In [90]:
model_checkpoint = "bert-base-cased"
tokenizer_checkpoint = "sgugger/bert-like-tokenizer"

We can apply the same tokenization function as before, we just need to update our tokenizer to use the checkpoint we just picked:

In [91]:
tokenizer = AutoTokenizer.from_pretrained(tokenizer_checkpoint)
tokenized_datasets = datasets.map(tokenize_function, batched=True, num_proc=4, remove_columns=["text"])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
loading file vocab.txt from cache at /root/.cache/huggingface/hub/models--sgugger--bert-like-tokenizer/snapshots/01644691345d776b01e25b0093968d2401a2310b/vocab.txt
loading file tokenizer.json from cache at /root/.cache/huggingface/hub/models--sgugger--bert-like-tokenizer/snapshots/01644691345d776b01e25b0093968d2401a2310b/tokenizer.json
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at /root/.cache/huggingface/hub/models--sgugger--bert-like-tokenizer/snapshots/01644691345d776b01e25b0093968d2401a2310b/special_tokens_map.json
loading file tokenizer_config.json from cache at /root/.cache/huggingface/hub/models--sgugger--bert-like-tokenizer/snapshots/01644691345

And like before, we group texts together and chunk them in samples of length `block_size`. You can skip that step if your dataset is composed of individual sentences.

In [92]:
lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=4,
)

The rest is very similar to what we had, with two exceptions. First we use a model suitable for masked LM:

In [93]:
from transformers import AutoConfig, AutoModelForMaskedLM

config = AutoConfig.from_pretrained(model_checkpoint)
model = AutoModelForMaskedLM.from_config(config)

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--bert-base-cased/snapshots/cd5ef92a9fb2f889e972770a36d4ed042daf221e/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.26.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 28996
}

Generate config GenerationConfig {
  "pad_token_id": 0,
  "transformers_version": "4.26.0"
}



We redefine our `TrainingArguments`:

In [94]:
training_args = TrainingArguments(
    "test-clm",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    push_to_hub=True,
    push_to_hub_model_id=f"{model_checkpoint}-wikitext2",
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1334: FutureWarning: `--push_to_hub_model_id` is deprecated and will be removed in version 5 of 🤗 Transformers. Use `--hub_model_id` instead and pass the full repo name to this argument (in this case acate/bert-base-cased-wikitext2).
  warnings.warn(


Like before, the last two arguments are to setup everything so we can push the model to the [Hub](https://huggingface.co/models) at the end of training. Remove the two of them if you didn't follow the installation steps at the top of the notebook, otherwise you can change the value of `push_to_hub_model_id` to something you would prefer.

Finally, we use a special `data_collator`. The `data_collator` is a function that is responsible of taking the samples and batching them in tensors. In the previous example, we had nothing special to do, so we just used the default for this argument. Here we want to do the random-masking. We could do it as a pre-processing step (like the tokenization) but then the tokens would always be masked the same way at each epoch. By doing this step inside the `data_collator`, we ensure this random masking is done in a new way each time we go over the data.

To do this masking for us, the library provides a `DataCollatorForLanguageModeling`. We can adjust the probability of the masking:

In [95]:
from transformers import DataCollatorForLanguageModeling
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)

Then we just have to pass everything to `Trainer` and begin training:

In [96]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_datasets["train"],
    eval_dataset=lm_datasets["validation"],
    data_collator=data_collator,
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'Repository' (from 'huggingface_hub.repository') is deprecated and will be removed from version '1.0'. Please prefer the http-based alternatives instead. Given its large adoption in legacy code, the complete removal is only planned on next major release.
For more details, please read https://huggingface.co/docs/huggingface_hub/concepts/git_vs_http.
  warnings.warn(warning_message, FutureWarning)
/content/test-clm is already a clone of https://huggingface.co/acate/bert-base-cased-wikitext2. Make sure you pull the latest changes with `repo.git_pull()`.


Download file runs/May23_00-05-18_1969c3de4f38/1716422817.0307846/events.out.tfevents.1716422817.1969c3de4f38.…

Download file runs/May23_00-05-18_1969c3de4f38/events.out.tfevents.1716426150.1969c3de4f38.1280.8: 100%|######…

Download file training_args.bin: 100%|##########| 3.87k/3.87k [00:00<?, ?B/s]

Download file runs/May23_00-05-18_1969c3de4f38/events.out.tfevents.1716422817.1969c3de4f38.1280.6: 100%|######…

Download file pytorch_model.bin:   0%|          | 32.0k/413M [00:00<?, ?B/s]

In [97]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 18761
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 7038
  Number of trainable parameters = 108340804
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss
1,7.091300,7.051738
2,6.905100,6.872865
3,6.881400,6.892736


Saving model checkpoint to test-clm/checkpoint-500
Configuration saved in test-clm/checkpoint-500/config.json
Configuration saved in test-clm/checkpoint-500/generation_config.json
Model weights saved in test-clm/checkpoint-500/pytorch_model.bin
Saving model checkpoint to test-clm/checkpoint-1000
Configuration saved in test-clm/checkpoint-1000/config.json
Configuration saved in test-clm/checkpoint-1000/generation_config.json
Model weights saved in test-clm/checkpoint-1000/pytorch_model.bin
Saving model checkpoint to test-clm/checkpoint-1500
Configuration saved in test-clm/checkpoint-1500/config.json
Configuration saved in test-clm/checkpoint-1500/generation_config.json
Model weights saved in test-clm/checkpoint-1500/pytorch_model.bin
Saving model checkpoint to test-clm/checkpoint-2000
Configuration saved in test-clm/checkpoint-2000/config.json
Configuration saved in test-clm/checkpoint-2000/generation_config.json
Model weights saved in test-clm/checkpoint-2000/pytorch_model.bin
***** Ru

TrainOutput(global_step=7038, training_loss=7.053426822229826, metrics={'train_runtime': 829.1075, 'train_samples_per_second': 67.884, 'train_steps_per_second': 8.489, 'total_flos': 3703423157830656.0, 'train_loss': 7.053426822229826, 'epoch': 3.0})

Like before, we can evaluate our model on the validation set. The perplexity is much lower than for the CLM objective because for the MLM objective, we only have to make predictions for the masked tokens (which represent 15% of the total here) while having access to the rest of the tokens. It's thus an easier task for the model.

In [40]:
eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

***** Running Evaluation *****
  Num examples = 2009
  Batch size = 8


The perplexity is still quite high since for this demo we trained on a small dataset for a small number of epochs. For a real LM training, you  would need a larger dataset and more epochs.

You can now upload the result of the training to the Hub, just execute this instruction:

In [98]:
trainer.push_to_hub()
tokenizer.push_to_hub("acate/bert-base-cased-wikitext2")

Saving model checkpoint to test-clm
Configuration saved in test-clm/config.json
Configuration saved in test-clm/generation_config.json
Model weights saved in test-clm/pytorch_model.bin


Upload file pytorch_model.bin:   0%|          | 1.00/413M [00:00<?, ?B/s]

Upload file runs/May23_02-07-38_1969c3de4f38/events.out.tfevents.1716430133.1969c3de4f38.1280.11:   0%|       …

To https://huggingface.co/acate/bert-base-cased-wikitext2
   c079638..10f2b06  main -> main

   c079638..10f2b06  main -> main

Dropping the following result as it does not have all the necessary fields:
{'task': {'name': 'Masked Language Modeling', 'type': 'fill-mask'}}
To https://huggingface.co/acate/bert-base-cased-wikitext2
   10f2b06..32d36f1  main -> main

   10f2b06..32d36f1  main -> main

tokenizer config file saved in bert-base-cased-wikitext2/tokenizer_config.json
Special tokens file saved in bert-base-cased-wikitext2/special_tokens_map.json
Uploading the following files to acate/bert-base-cased-wikitext2: tokenizer.json,tokenizer_config.json,vocab.txt,special_tokens_map.json


CommitInfo(commit_url='https://huggingface.co/acate/bert-base-cased-wikitext2/commit/1ff237a73049a3709b2a3479f0479bfcccdc9fbe', commit_message='Upload tokenizer', commit_description='', oid='1ff237a73049a3709b2a3479f0479bfcccdc9fbe', pr_url=None, pr_revision=None, pr_num=None)

You can now share this model with all your friends, family, favorite pets: they can all load it with the identifier `"your-username/the-name-you-picked"` so for instance:

```python
from transformers import AutoModelForMaskedLM

model = AutoModelForMaskedLM.from_pretrained("sgugger/my-awesome-model")
```